In [ ]:
from d123.dataset.scene.scene_builder import ArrowSceneBuilder
from d123.dataset.scene.scene_filter import SceneFilter

from nuplan.planning.utils.multithreading.worker_sequential import Sequential
from d123.common.datatypes.sensor.camera import CameraType

In [ ]:
# split = "nuplan_private_test"
# log_names = ["2021.09.29.17.35.58_veh-44_00066_00432"]


splits = ["wopd_train"]
# splits = ["carla"]
# splits = ["nuplan_private_test"]
# log_names = None



log_names = None
scene_tokens = None

scene_filter = SceneFilter(
    split_names=splits,
    log_names=log_names,
    scene_tokens=scene_tokens,
    duration_s=19,
    history_s=0.0,
    timestamp_threshold_s=20,
    shuffle=True,
    camera_types=[CameraType.CAM_F0],
)
scene_builder = ArrowSceneBuilder("/home/daniel/d123_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print(f"Found {len(scenes)} scenes")

In [ ]:
from typing import List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
from d123.common.geometry.base import Point2D
from d123.common.visualization.color.color import BLACK, DARK_GREY, DARKER_GREY, LIGHT_GREY, NEW_TAB_10, TAB_10
from d123.common.visualization.color.config import PlotConfig
from d123.common.visualization.color.default import CENTERLINE_CONFIG, MAP_SURFACE_CONFIG, ROUTE_CONFIG
from d123.common.visualization.matplotlib.observation import (
    add_box_detections_to_ax,
    add_default_map_on_ax,
    add_ego_vehicle_to_ax,
    add_traffic_lights_to_ax,
)
from d123.common.visualization.matplotlib.utils import add_shapely_linestring_to_ax, add_shapely_polygon_to_ax
from d123.dataset.maps.abstract_map import AbstractMap
from d123.dataset.maps.abstract_map_objects import AbstractLane
from d123.dataset.maps.map_datatypes import MapSurfaceType
from d123.dataset.scene.abstract_scene import AbstractScene


import shapely.geometry as geom

LEFT_CONFIG: PlotConfig = PlotConfig(
    fill_color=BLACK,
    fill_color_alpha=1.0,
    line_color=TAB_10[2],
    line_color_alpha=0.5,
    line_width=1.0,
    line_style="-",
    zorder=3,
)

RIGHT_CONFIG: PlotConfig = PlotConfig(
    fill_color=BLACK,
    fill_color_alpha=1.0,
    line_color=TAB_10[3],
    line_color_alpha=0.5,
    line_width=1.0,
    line_style="-",
    zorder=3,
)


LANE_CONFIG: PlotConfig = PlotConfig(
    fill_color=LIGHT_GREY,
    fill_color_alpha=1.0,
    line_color=BLACK,
    line_color_alpha=0.0,
    line_width=0.0,
    line_style="-",
    zorder=1,
)

ROAD_EDGE_CONFIG: PlotConfig = PlotConfig(
    fill_color=DARKER_GREY.set_brightness(0.0),
    fill_color_alpha=1.0,
    line_color=DARKER_GREY.set_brightness(0.0),
    line_color_alpha=1.0,
    line_width=1.0,
    line_style="-",
    zorder=3,
)

ROAD_LINE_CONFIG: PlotConfig = PlotConfig(
    fill_color=DARKER_GREY,
    fill_color_alpha=1.0,
    line_color=NEW_TAB_10[5],
    line_color_alpha=1.0,
    line_width=1.5,
    line_style="-",
    zorder=3,
)



def add_debug_map_on_ax(
    ax: plt.Axes,
    map_api: AbstractMap,
    point_2d: Point2D,
    radius: float,
    route_lane_group_ids: Optional[List[int]] = None,
) -> None:
    layers: List[MapSurfaceType] = [
        MapSurfaceType.LANE,
        MapSurfaceType.LANE_GROUP,
        MapSurfaceType.GENERIC_DRIVABLE,
        MapSurfaceType.CARPARK,
        MapSurfaceType.CROSSWALK,
        MapSurfaceType.INTERSECTION,
        MapSurfaceType.WALKWAY,
        MapSurfaceType.ROAD_EDGE,
        # MapSurfaceType.ROAD_LINE,
    ]
    x_min, x_max = point_2d.x - radius, point_2d.x + radius
    y_min, y_max = point_2d.y - radius, point_2d.y + radius
    patch = geom.box(x_min, y_min, x_max, y_max)
    map_objects_dict = map_api.query(geometry=patch, layers=layers, predicate="intersects")

    edge_lengths = []
    for layer, map_objects in map_objects_dict.items():
        for map_object in map_objects:
            try:
                if layer in [
                    MapSurfaceType.GENERIC_DRIVABLE,
                    MapSurfaceType.CARPARK,
                    # MapSurfaceType.CROSSWALK,
                    MapSurfaceType.INTERSECTION,
                    # MapSurfaceType.WALKWAY,
                ]:
                    add_shapely_polygon_to_ax(ax, map_object.shapely_polygon, MAP_SURFACE_CONFIG[layer])

                if layer in [MapSurfaceType.LANE_GROUP]:
                    add_shapely_polygon_to_ax(ax, map_object.shapely_polygon, MAP_SURFACE_CONFIG[layer])

                if layer in [MapSurfaceType.LANE]:
                    map_object: AbstractLane
                    add_shapely_linestring_to_ax(ax, map_object.centerline.linestring, CENTERLINE_CONFIG)
                    # add_shapely_linestring_to_ax(ax, map_object.right_boundary.linestring, RIGHT_CONFIG)
                    # add_shapely_linestring_to_ax(ax, map_object.left_boundary.linestring, LEFT_CONFIG)
                    # add_shapely_polygon_to_ax(ax, map_object.shapely_polygon, LANE_CONFIG)

                    # centroid = map_object.shapely_polygon.centroid
                    # ax.text(
                    #     centroid.x,
                    #     centroid.y,
                    #     str(map_object.id),
                    #     horizontalalignment="center",
                    #     verticalalignment="center",
                    #     fontsize=8,
                    #     bbox=dict(facecolor="white", alpha=0.7, boxstyle="round,pad=0.2"),
                    # )
                if layer in [MapSurfaceType.ROAD_EDGE]:
                    add_shapely_linestring_to_ax(ax, map_object.polyline_3d.linestring, ROAD_EDGE_CONFIG)
                    edge_lengths.append(map_object.polyline_3d.linestring.length)

                # if layer in [MapSurfaceType.ROAD_LINE]:
                #     add_shapely_linestring_to_ax(ax, map_object.polyline_3d.linestring, ROAD_LINE_CONFIG)

            except Exception:
                import traceback

                print(f"Error adding map object of type {layer.name} and id {map_object.id}")
                traceback.print_exc()

    print(f"road edge mean: {np.mean(edge_lengths)} m")
    print(f"road edge min: {np.min(edge_lengths)} m")
    print(f"road edge max: {np.max(edge_lengths)} m")
    ax.set_title(f"Map: {map_api.map_name}")


def _plot_scene_on_ax(ax: plt.Axes, scene: AbstractScene, iteration: int = 0, radius: float = 80) -> plt.Axes:

    ego_vehicle_state = scene.get_ego_state_at_iteration(iteration)
    box_detections = scene.get_box_detections_at_iteration(iteration)

    point_2d = ego_vehicle_state.bounding_box.center.state_se2.point_2d
    add_debug_map_on_ax(ax, scene.map_api, point_2d, radius=radius, route_lane_group_ids=None)
    # add_default_map_on_ax(ax, scene.map_api, point_2d, radius=radius, route_lane_group_ids=None)
    # add_traffic_lights_to_ax(ax, traffic_light_detections, scene.map_api)

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax(ax, ego_vehicle_state)

    zoom = 1.0
    ax.set_xlim(point_2d.x - radius * zoom, point_2d.x + radius * zoom)
    ax.set_ylim(point_2d.y - radius * zoom, point_2d.y + radius * zoom)

    ax.set_aspect("equal", adjustable="box")
    return ax


def plot_scene_at_iteration(
    scene: AbstractScene, iteration: int = 0, radius: float = 80
) -> Tuple[plt.Figure, plt.Axes]:

    size = 15

    fig, ax = plt.subplots(figsize=(size, size))
    _plot_scene_on_ax(ax, scene, iteration, radius)
    return fig, ax


scene_index = 8
fig, ax = plot_scene_at_iteration(scenes[scene_index], iteration=100, radius=40)

# fig.savefig(f"/home/daniel/scene_{scene_index}_iteration_1.pdf", dpi=300, bbox_inches="tight")